In [64]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [65]:
df = pd.read_csv('metadata_3d.csv')
df.head()

,patient_id,nodule_idx,malignancy,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LIDC-IDRI-0001,0,5.0,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,LIDC-IDRI-0002,0,4.5,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,LIDC-IDRI-0003,0,2.0,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,LIDC-IDRI-0003,1,4.5,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,LIDC-IDRI-0003,2,3.5,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


In [66]:
df.drop("patient_id", axis=1, inplace=True)
df.drop("nodule_idx", axis=1, inplace=True)
df.drop("is_cancer", axis=1, inplace=True)
#df.head()

In [67]:
df = df.loc[:, df.nunique() > 1]
df = df.loc[:,~df.T.duplicated()] # df.T e a transposta, depois trata as colunas como linhas para encontrar repetidas
df.head()

,malignancy,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,5.0,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,26.977005,34.406945,6652.204514,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,4.5,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,37.724947,40.384819,8224.320213,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,2.0,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,25.552264,32.630357,4221.259693,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,4.5,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,24.750701,30.922607,5397.873974,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,3.5,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,12.934109,13.267970,422.392330,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


In [68]:
correlation_matrix = df.corr()
malignancy_corr = correlation_matrix['malignancy']

strong_corr_with_malignancy = malignancy_corr[(malignancy_corr > 0.55) | (malignancy_corr < -0.55)]

strong_corr_with_malignancy = strong_corr_with_malignancy.drop('malignancy') 
print('existem', strong_corr_with_malignancy.shape[0], 'colunas com uma correlacao naquele intervalo')

existem 19 colunas com uma correlacao naquele intervalo


In [69]:
strong_corr_columns = strong_corr_with_malignancy.index # nome das colunas que sobraram

subset_df = df[strong_corr_columns] # subset do dataset que tem estas colunas

subset_df = pd.concat([df[['malignancy']], subset_df], axis=1) # voltar a adicionar (no inicio)

subset_df.head()


,malignancy,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MinorAxisLength,original_shape_SurfaceVolumeRatio,original_glcm_Id,original_glcm_Idm,original_glcm_Imc1,original_glcm_MCC,original_gldm_DependenceNonUniformityNormalized,original_gldm_SmallDependenceEmphasis,original_glrlm_RunLengthNonUniformityNormalized,original_glrlm_RunPercentage,original_glrlm_ShortRunEmphasis,original_glszm_ZonePercentage,original_ngtdm_Strength
0,5.0,19.588162,22.827597,31.038346,32.075141,26.977005,34.406945,22.206076,0.368609,0.357261,0.288097,-0.139251,0.533478,0.142832,0.322797,0.814938,0.881115,0.918743,0.363301,1.274217
1,4.5,21.632902,28.944112,36.659875,30.945300,37.724947,40.384819,23.027779,0.388414,0.453400,0.382826,-0.206279,0.823689,0.105000,0.147483,0.738580,0.842797,0.882628,0.141313,0.926980
2,2.0,14.561133,26.516200,26.873199,31.855156,25.552264,32.630357,20.380759,0.472904,0.376161,0.295529,-0.138308,0.627493,0.154051,0.255768,0.830722,0.902953,0.927897,0.282061,4.007826
3,4.5,16.647824,24.114835,26.959597,30.922607,24.750701,30.922607,20.180327,0.361115,0.340451,0.267210,-0.133980,0.612321,0.168861,0.354250,0.829927,0.893385,0.927213,0.405739,1.578325
4,3.5,7.316637,11.451529,11.145619,12.548361,12.934109,13.267970,7.644348,0.816013,0.195261,0.120869,-0.386747,0.707442,0.356263,0.588421,0.942074,0.967286,0.976638,0.662835,5.874138
